<a href="https://colab.research.google.com/github/GiuliaLanzillotta/exercises/blob/master/Tweet_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation of **Generating sentences from a continuous space paper**
Here's a link to the [paper](https://arxiv.org/pdf/1511.06349v4.pdf).


> ### The goal 
 What I want to explore here is the expression of sentiment in generative models. <br>
 The dataset consists of two different samples of tweets, one with positive sentiment and one with negative sentiment. <br>
 The goal is to train two generators on the two sets separetly and analyse the qualitative differences.

# Data loading

In [0]:
!ls

drive  sample_data  twitter-datasets


In [0]:
!unzip 'drive/My Drive/twitter-datasets.zip'

Archive:  drive/My Drive/twitter-datasets.zip
  inflating: twitter-datasets/sample_submission.csv  
  inflating: twitter-datasets/test_data.txt  
  inflating: twitter-datasets/train_neg_full.txt  
  inflating: twitter-datasets/train_neg.txt  
  inflating: twitter-datasets/train_pos_full.txt  
  inflating: twitter-datasets/train_pos.txt  


In [0]:
positive_location = 'twitter-datasets/train_pos.txt'
negative_location = 'twitter-datasets/train_neg.txt'

An example of the raw data 

In [0]:
!head -3 'twitter-datasets/train_pos.txt'

<user> i dunno justin read my mention or not . only justin and god knows about that , but i hope you will follow me #believe 15
because your logic is so dumb , i won't even crop out your name or your photo . tsk . <url>
" <user> just put casper in a box ! " looved the battle ! #crakkbitch


In [0]:
!wc -l 'twitter-datasets/train_pos.txt'


100000 twitter-datasets/train_pos.txt


# Text preprocessing

In [0]:
from nltk.tokenize.casual import TweetTokenizer
from collections import Counter
import pickle

In [0]:
def tokenize_text(text):
    """
    Transforms the specified files in tokens using the Twitter tokenizer.
    @:params: str
        Input text to tokenize
    @:return: list(str)
        Returns the tokens as a list of strings.
    """
    tokenizer = TweetTokenizer()
    # tokenizing the text
    tokens = tokenizer.tokenize(text)
    words = [w.lower() for w in tokens]
    return words

Now we build a vocabulary with the most frequent words

In [307]:
frequency_treshold = 100000*0.001 # 0.1% of the tweets should contain the "frequent words"
frequency_treshold

100.0

In [0]:
def build_vocabulary(file_name, output_file_name):
  """
  Builds the vocabulary for the specified file.
  """
  words = []
  print("Reading ",file_name)
  raw = open(file_name,  "r").read()
  more_words = tokenize_text(raw)
  words.extend(more_words)
  # counting the words
  counter = Counter(words)
  words_count = dict(counter)
  # filtering 
  filtered_words = [k for k, v in words_count.items() if v >= frequency_treshold]
  # building voabulary 
  # index starts at 1 so that the 0 can be left for the empty spaces
  vocab = {k:i+1 for i,k in enumerate(filtered_words)}
  # saving the vocabulary 
  with open(output_file_name, 'wb') as f: 
      pickle.dump(vocab, f, pickle.HIGHEST_PROTOCOL)
  return vocab

In [309]:
vocab_pos = build_vocabulary(positive_location, "vocab_pos.pkl")
vocab_neg = build_vocabulary(negative_location, "vocab_neg.pkl")

Reading  twitter-datasets/train_pos.txt
Reading  twitter-datasets/train_neg.txt


In [310]:
len(vocab_pos.keys())

1082

In [311]:
len(vocab_neg.keys())

1545

In [0]:
def filter_sentence(sentence, vocab):
  """
  Filters the given sentence with the given vocabulary. 
    The words that are not in the vocabulary will be filtered out 
    of the sentence. 
  """
  return [word for word in sentence if word in vocab.keys()]

In [0]:
def sentence_to_sequence(sentence, vocab):
  """
  (Filtering included)
  Transforms the given sentence into a sequence of ints, 
  corresponding to the index of the word.
  """
  filtered_sentence = filter_sentence(sentence, vocab)
  return [vocab[word] for word in filtered_sentence]

# Sentences handler
During training we want to be able to load the tweets from the file sequentially for memory efficiency. <br>
Here I implement the helper functions that will do that


In [0]:
def load_new_chunk(start, chunksize, file):
  """
  Loads -chunksize- tweets from the specified file, starting from 
  -start- tweet (excluded). 
  """
  f=open(file)
  lines=f.readlines()
  selected = lines[start:start+chunksize]
  return selected 

In [315]:
load_new_chunk(99997, 5, positive_location)

["<user> <user> um gord ... i just read your profile . i'm not sure i can have lunch with a riders fan\n",
 "<user> i'm so excited for tomorrow ! look out for two leprechauns ! xx\n",
 'i always wondered what the job application is like at hooters .. do they just give you a bra and say , " here fill this out " .. ? "\n']

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
def input_generator(file, start, chunksize, max_seq_length, vocab):
  """
  The output of the generator must be a tuple (inputs, targets)
  This tuple (a single output of the generator) makes a single batch
  Different batches may have different sizes. For example, 
  the last batch of the epoch is commonly smaller than the others, 
  if the size of the dataset is not divisible by the batch size. 
  """
  sentences = load_new_chunk(start,chunksize,file)
  sequences = [sentence_to_sequence(sentence,vocab) for sentence in sentences]
  padded_sequences = pad_sequences(sequences, maxlen=max_seq_length, padding="pre", value=0)
  return padded_sequences

In [318]:
input_generator(positive_location, 0, 5, 100, vocab_pos)

array([[   0,    0,    0,    0,  659,  101,  225, 1017,  397,   43,    2,
         478,  101,  889,  889,  689,  101,  225,  494,    2,  889,  397,
        1017,   38,  478,  885,  795,  885, 1017,  889,  494,    2,  689,
         889,  689,  397,  889,  689,  494,    9,  689,  889,  881,  795,
         101,  225,  494,    2,  889,   38,  889,  478, 1034,  689,  478,
         304,  889,  689,  713,  225,   38,  448,  689,  101,  494,  494,
          38,  494,   16,  448,  101,  494,    2,  689,  592, 1017,  795,
         689,  101,  713,    2,  881,  881, 1041,  689,  881,  881,  689,
         713,  885, 1017, 1006,  448, 1017,  881,    2, 1017, 1017,  151,
         610],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,  448, 1017,
         348,   38,  101,  225, 1017,  795,  689,  101,  397,  881,  689,
        1034,    2,  34

# Embeddings 

To embed the words we are going to use a pre-trained GloVe model as a *prior* on the embedding that we'll be inserted as a first layer to the final model.

In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-04-14 12:44:37--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-04-14 12:44:37--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-04-14 12:44:38--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


We are going to use the *200d* embedding file 

In [0]:
glove_location = 'glove.6B.200d.txt'

In [0]:
import numpy as np

In [0]:
embeddings_index = {}
f = open(glove_location, encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [0]:
def build_emb_matrix(glove, vocab):
  """
  Build the pre-trained GloVe embedding on the given vocabulary. 
  The GloVe embedding should be presented as a vocabulary. 
  """
  print("Building the GloVe embedding.")
  N = len(vocab.keys()) + 1 # number of words in the vocabulary 
  D = 200 # embedding dimension
  emb_matrix = np.zeros((N,D))
  for idx, word in enumerate(vocab.keys()):
    embedding_vector = glove.get(word)
    if embedding_vector is not None: emb_matrix[idx+1] = embedding_vector
    else: emb_matrix[idx+1] = embeddings_index.get('unk')
  return emb_matrix

In [0]:
emb_matrix_pos = build_emb_matrix(embeddings_index, vocab_pos)
emb_matrix_neg = build_emb_matrix(embeddings_index, vocab_neg)

Building the GloVe embedding.
Building the GloVe embedding.


In [0]:
emb_matrix_pos.shape

(1083, 200)

In [0]:
np.savez("emb_matrix_pos",emb_matrix_pos)
np.savez("emb_matrix_neg",emb_matrix_neg)

In [0]:
emb_matrix_pos = np.load("emb_matrix_pos.npz")["arr_0"]
emb_matrix_neg = np.load("emb_matrix_neg.npz")["arr_0"]

# VAE model
To make my life simpler I chose to use Keras to build the model. 

In [0]:
from keras.layers.advanced_activations import ELU

#### Hyperparameters first. 

In [0]:
VALIDATION_SPLIT = 0.3
MAX_SEQUENCE_LENGTH = 100
BATCH_SIZE = 100

In [0]:
LATENT_DIM = 64
HIDDEN_DIM = 96

#### text VAE class

In the following cells we'll build a **Model** class and its **Layers**' classes

In [0]:
import tensorflow as tf
from tensorflow.keras import layers

In [0]:
class Sampling(layers.Layer):
  """
  Sampling layer. 
  The sampling will be from the posterior on the latent vector, 
  whose prior is a standard Gaussian. 
  """
  def call(self, inputs):
    z_mean, z_log_var = inputs
    # batch and latent dimensionality
    N= tf.shape(z_mean)[0] 
    D = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(N, D))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon


In [0]:
class Embedding(layers.Layer):
  """
  Embedding layer. 
  Responsible for training the embedding. 
  """
  def __init__(self,
               vocab_dim, # (number of words x embemdding dimension)
               seq_length, #  maximum seq_length
               emb_matrix= None, # pre-trained embeddings
               name='embedding',
               dynamic=True,
               to_train= True, # wether to train the embeddings
               **kwargs):
    super(Embedding, self).__init__(name=name, **kwargs)
    N, D = vocab_dim
    self.embedding = layers.Embedding(N, D, weights=[emb_matrix],
                                      input_length=seq_length, trainable=to_train)

  def call(self, inputs):
    x = self.embedding(inputs)
    return x

In [0]:
class Encoder(layers.Layer):
  """
  Encoder layer. 
  This layer parametrizes the mapping from the input  
  to a latent space distribution. 
  Concretely it will map sentences to the triple (z_mean, z_log_var, z).
  """
  
  def __init__(self,
               vocab_dim, # (number of words x embemdding dimension)
               seq_length, #  maximum seq_length
               latent_dim=LATENT_DIM,
               hidden_dim=HIDDEN_DIM,
               name='encoder',
               dropout_rate=0.2,
               dynamic=True,
               activation_fun='elu',
               **kwargs):
    super(Encoder, self).__init__(name=name, **kwargs)
    N, D = vocab_dim
    self.recurrent_layer = layers.Bidirectional(layers.LSTM(hidden_dim, 
                            return_sequences=False, recurrent_dropout=dropout_rate,
                            input_shape=(None,seq_length,D)), merge_mode='concat')
    self.dense_proj = layers.Dense(hidden_dim, activation=activation_fun)
    self.dense_mean = layers.Dense(latent_dim)
    self.dense_log_var = layers.Dense(latent_dim)
    self.sampling = Sampling()

  def call(self, inputs):
    # Input must be of shape (None, max_len)
    h = self.recurrent_layer(inputs)
    x = self.dense_proj(h)
    z_mean = self.dense_mean(x)
    z_log_var = self.dense_log_var(x)
    z = self.sampling((z_mean, z_log_var))
    return z_mean, z_log_var, z

In [0]:
class Decoder(layers.Layer):
  """
  Decoder layer. 
  This layer parametrizes the mapping from the latent space to the 
  output dimension. 
  Concretely it will map a z to a readable sentence.
  """
  def __init__(self,
               vocab_dim, # (number of words x embemdding dimension)
               seq_length, #  maximum seq_length
               hidden_dim=HIDDEN_DIM,
               name='decoder',
               dynamic=True,
               dropout_rate=0.2,
               **kwargs):
    super(Decoder, self).__init__(name=name, **kwargs)
    N, D = vocab_dim
    self.repeat_layer = layers.RepeatVector(seq_length)
    self.recurrent_layer = layers.LSTM(hidden_dim, return_sequences=True, 
                                       recurrent_dropout=dropout_rate, 
                                       input_shape=(None,seq_length,D))
    self.mean = layers.TimeDistributed(layers.Dense(N, activation='linear'))

  def call(self, inputs):
    z_repeated = self.repeat_layer(inputs)
    h_decoded = self.recurrent_layer(z_repeated)
    x_decoded_mean = self.mean(h_decoded)
    return h_decoded, x_decoded_mean

In [0]:
from tensorflow.keras import backend as K
import tensorflow_addons as tfa

In [0]:
class textVAE(tf.keras.Model):
  """
  Model class for VAE.
  Combines the embedding, encoder and decoder into an 
  end-to-end model for training."""

  def __init__(self,
               vocab_dim, # (number of words x embemdding dimension)
               seq_length, #  maximum seq_length
               emb_matrix=None, # pre-trained embeddings - if None, they will be trained
               train_emb = True,
               hidden_dim=HIDDEN_DIM,
               latent_dim=LATENT_DIM,
               dropout_rate=0.2,
               name='textVAE',
               **kwargs):
    super(textVAE, self).__init__(name=name, **kwargs)
    self.N, self.D = vocab_dim
    self.seq_length = seq_length
    self.embedding = Embedding(vocab_dim=vocab_dim,
                               seq_length=seq_length,
                               emb_matrix=emb_matrix,
                               to_train=train_emb)
    self.encoder = Encoder(vocab_dim=vocab_dim,
                           seq_length=seq_length,
                           latent_dim=latent_dim,
                           hidden_dim=hidden_dim, 
                           dropout_rate=dropout_rate)
    self.decoder = Decoder(vocab_dim=vocab_dim, 
                           seq_length=seq_length,
                           hidden_dim=hidden_dim, 
                           dropout_rate=dropout_rate)
    
  def compute_loss(self, z_mean, z_log_var, z, 
                   x, x_decoded_mean, target_weights):
    """
    Computes the VAE loss. 
    """
    labels = tf.cast(x, tf.int32)
    xent_loss = K.sum(tfa.seq2seq.sequence_loss(x_decoded_mean, 
                        labels, weights=target_weights, 
                        average_across_timesteps=False,
                        average_across_batch=False), axis=-1)
    kl_loss = - 0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - 
                                     tf.exp(z_log_var) + 1)
    return K.mean(xent_loss + kl_loss)


  def call(self, inputs):
    x = self.embedding(inputs)
    z_mean, z_log_var, z = self.encoder(x)
    h_decoded,x_decoded_mean = self.decoder(z)
    # Add KL divergence regularization loss.
    target_weights=tf.ones_like(inputs, dtype=tf.float32)
    loss = self.compute_loss(z_mean, z_log_var, z, 
                             inputs, x_decoded_mean, target_weights) 
    self.add_loss(loss)
    return h_decoded,x_decoded_mean  

  def predict_latent_mean(self,inputs):
    """
    Uses the encoder part of the trained generator to go 
    from the input space to the latent space.
    """
    x = self.embedding(inputs)
    z_mean, _, _ = self.encoder(x)
    return z_mean
  
  def predict_sentence(self,inputs):
    """
    Uses the decoder part of the trained generator to 
    go from the latent space to a new sentence.
    """
    _,x_decoded_mean = self.decoder(inputs)
    return x_decoded_mean

In [0]:
LEARNING_RATE = 1e-3

In [0]:
def construct_model(emb_matrix, name):
  """
  Constructing the text VAE model.
  """
  N,D = emb_matrix.shape
  vae = textVAE(vocab_dim=(N,D),
                seq_length=MAX_SEQUENCE_LENGTH,
                emb_matrix=emb_matrix, 
                name=name)

  optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

  vae.compile(optimizer=optimizer)
  return vae

In [0]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [335]:
generator_pos = construct_model(emb_matrix_pos, name="pos_vae")
generator_pos

# Training 

In [0]:
model_checkpoint_file_pos = "textVAE_pos_checkpt"
model_checkpoint_file_neg = "textVAE_neg_checkpt"

In [0]:
EPOCHS = 1
N_STEPS = int(100000/BATCH_SIZE) 
SAVING_STEPS = [300,700,999]

In [0]:
def train_model(model, input_file_location, vocab, model_location, 
                epochs=EPOCHS, n_steps=N_STEPS, batch_size=BATCH_SIZE):
  """
  Trains the given model for the given number of epochs, 
  making -n_steps- in each epoch. 
  """
  for epoch in range(EPOCHS):
    print('-------epoch: ',epoch,'--------')
    for step in range(N_STEPS):
      start = step*BATCH_SIZE
      input_sequences = input_generator(positive_location,start=start, chunksize=BATCH_SIZE,
                                        max_seq_length=MAX_SEQUENCE_LENGTH, vocab=vocab)
      model.fit(x=input_sequences, y=input_sequences, batch_size = batch_size, epochs=1, 
                validation_split=VALIDATION_SPLIT, dynamic=True)
      if step in SAVING_STEPS: 
        loss = model.losses
        print('----step: ',step,' ----loss:', loss)
        model.save(model_location)
  
  model.save(model_location)
  return model

In [343]:
generator_pos_trained = train_model(generator_pos, positive_location, vocab_pos, 
                                    model_checkpoint_file_pos)

-------epoch:  0 --------
1/1 [==============================] - 0s 111ms/step - loss: 174.8709 - val_loss: 179.2274
----step:  300  ----loss: [<tf.Tensor 'pos_vae/Mean_1:0' shape=() dtype=float32>]
1/1 [==============================] - 0s 105ms/step - loss: 165.1978 - val_loss: 170.9005
----step:  700  ----loss: [<tf.Tensor 'pos_vae/Mean_1:0' shape=() dtype=float32>]
1/1 [==============================] - 0s 99ms/step - loss: 168.2019 - val_loss: 174.4834
----step:  999  ----loss: [<tf.Tensor 'pos_vae/Mean_1:0' shape=() dtype=float32>]


In [344]:
generator_neg = construct_model(emb_matrix_neg, name="neg_vae")
generator_neg_trained = train_model(generator_neg, negative_location, vocab_neg, 
                                    model_checkpoint_file_neg)

-------epoch:  0 --------
1/1 [==============================] - 0s 96ms/step - loss: 176.3483 - val_loss: 181.3461
----step:  300  ----loss: [<tf.Tensor 'neg_vae/Mean_1:0' shape=() dtype=float32>]
1/1 [==============================] - 0s 96ms/step - loss: 179.1386 - val_loss: 185.1933
----step:  700  ----loss: [<tf.Tensor 'neg_vae/Mean_1:0' shape=() dtype=float32>]
1/1 [==============================] - 0s 100ms/step - loss: 180.1057 - val_loss: 193.0202
----step:  999  ----loss: [<tf.Tensor 'neg_vae/Mean_1:0' shape=() dtype=float32>]


# Tests 
We are now going to use the trained model to do sentence interpolation, but we need a few helper functions to do so 

In [0]:
index2word_pos = {v:k for k,v in vocab_pos.items()}
index2word_neg = {v:k for k,v in vocab_neg.items()}

In [0]:
def print_sampled_sentence(sentence_vector, model, index2word, latent_dim=LATENT_DIM, max_len=MAX_SEQUENCE_LENGTH):
  """
  Uses the trained model
  @:param: sentence vector: latent space representation of a sentence
      (this should be the output of predict_latent_mean)

  """
  N = len(index2word.keys())+1
  sentence_vector = np.reshape(sentence_vector,[1,latent_dim]) # reshaping into 1 x latent space, where 1 is the batch size
  generated = tf.keras.activation.softmax(model.predict_sentence(sentence_vector))
  generated = np.reshape(generated,[max_len,N]) # reshaping into sequence length x vocabulary words 
  generated_indices = np.apply_along_axis(np.argmax, 1, generated)
  word_list = list(np.vectorize(index2word.get)(generated_indices))
  w_list = [w for w in word_list if w] # filtering out the words
  print(' '.join(w_list))

In [0]:
def shortest_homologies(point1,point2,n):
  """
  Discover n mid-way  points in the path between point1 and point2.
  The name of the functions is due to the fact that the points are in the 
  latent space.
  """
  dist_vec = point2 - point1
  sample = np.linspace(0, 1, n, endpoint = True)
  samples = []
  for s in sample:
      samples.append(point1 + s * dist_vec)
  return samples

In [0]:
def sentences_interpolation(sentence1, sentence2, n, 
                            vocab, model, index2word, 
                            max_len=MAX_SEQUENCE_LENGTH):
  """
  Interpolating between the two given sentences in n steps.
  """
  sequence1 = sentence_to_sequence(sentence1, vocab)
  sequence1 = pad_sequences(sequence1, maxlen=max_len, padding="pre", value=0)
  sequence2 = sentence_to_sequence(sentence2, vocab)
  sequence2 = pad_sequences(sequence2, maxlen=max_len, padding="pre", value=0)
  latent1 = model.predict_latent_mean(sequence1)
  latent2 = model.predict_latent_mean(sequence2)
  homologies = shortest_homologies(latent1, latent2, n)
  for latent_sentence in homologies:
    print_sampled_sentence(latent_sentence, model, index2word)

In [374]:
sentence1 = ["I think machine learning is great"]
sentence2 = ["Do you want a new book"]
print("Positive sentence interpolation")
sentences_interpolation(sentence1,sentence2,10,
                        vocab_pos,generator_pos, index2word_pos)
print("Negative sentence interpolation")
sentences_interpolation(sentence1,sentence2,10,
                        vocab_neg,generator_neg, index2word_neg)

Positive sentence interpolation
tf.Tensor([], shape=(0, 64), dtype=float32)
[<tf.Tensor: shape=(0, 64), dtype=float32, numpy=array([], shape=(0, 64), dtype=float32)>, <tf.Tensor: shape=(0, 64), dtype=float32, numpy=array([], shape=(0, 64), dtype=float32)>, <tf.Tensor: shape=(0, 64), dtype=float32, numpy=array([], shape=(0, 64), dtype=float32)>, <tf.Tensor: shape=(0, 64), dtype=float32, numpy=array([], shape=(0, 64), dtype=float32)>, <tf.Tensor: shape=(0, 64), dtype=float32, numpy=array([], shape=(0, 64), dtype=float32)>, <tf.Tensor: shape=(0, 64), dtype=float32, numpy=array([], shape=(0, 64), dtype=float32)>, <tf.Tensor: shape=(0, 64), dtype=float32, numpy=array([], shape=(0, 64), dtype=float32)>, <tf.Tensor: shape=(0, 64), dtype=float32, numpy=array([], shape=(0, 64), dtype=float32)>, <tf.Tensor: shape=(0, 64), dtype=float32, numpy=array([], shape=(0, 64), dtype=float32)>, <tf.Tensor: shape=(0, 64), dtype=float32, numpy=array([], shape=(0, 64), dtype=float32)>]


ValueError: ignored